<a href="https://colab.research.google.com/github/jpellicott/r_freightbrokers_sentiment/blob/main/NER_and_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair

     |████████████████████████████████| 319 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9 MB 29.8 MB/s 
     |████████████████████████████████| 1.2 MB 20.9 MB/s 
     |████████████████████████████████| 788 kB 46.3 MB/s 
     |████████████████████████████████| 19.7 MB 38.1 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 981 kB 47.2 MB/s 
     |████████████████████████████████| 48 kB 6.2 MB/s 
     |████████████████████████████████| 56 kB 5.5 MB/s 
     |████████████████████████████████| 62 kB 916 kB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
     |████████████████████████████████| 636 kB 46.9 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=21d4deaf994a6870dd6ec7b2beeacece6ddb666eaf5d0

In [ ]:
import pandas as pd
import flair

In [ ]:
model = flair.models.TextClassifier.load('en-sentiment')


2021-10-07 00:52:03,499 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [ ]:

def get_sentiment(text):
    # tokenize input text
    sentence = flair.data.Sentence(text)
    # make sentiment prediction
    model.predict(sentence)
    # extract sentiment direction and confidence (label and score) object
    sentiment = sentence.labels[0]
    return sentiment

In [ ]:
# load data
# data is from the Freighbrokers subreddit
df = pd.read_csv('/content/new_reddit_freight_brokers_ner.csv', sep='|')
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,gpe
0,1.633555e+09,0.0,t3_q2tok0,1.0,"USDOT Number: 3300318, MC-1046672 \n\nThinking of taking my talents elsewhere and lease to them. What they are saying sounds great but you have to hear it from the people who have say to load the truck or not.\n\nSo what’s your point of view?",FreightBrokers,What do you guys think of Cloud Trucks,1.0,1.00,[]
1,1.633554e+09,0.0,t3_q2teic,1.0,Does anyone use this load board app? I recently starting using it but idk how trustworthy it is,FreightBrokers,Doft?,1.0,1.00,[]
2,1.633546e+09,0.0,t3_q2qvl2,3.0,"I’ve have been applying to various 3PLs in hopes of getting hired as a “carrier sales rep” as most companies call it. I’ve applied to several and none have gotten back to me in almost a week... except for one. I applied to a “certain” company with a... less than stellar reputation around here, and they responded. Now, should they offer me a position I’ll be tempted to take the job and do what I can to learn the business before moving on.\n\nI just wanted to hear you guy’s thoughts on this. Would I be shooting myself in the foot? Should I wait longer to hear back from other companies? Or will everything be fine if I do my best, put in my time, and move on?",FreightBrokers,I guess this might be my only option?,3.0,0.81,[]
3,1.633529e+09,0.0,t3_q2ky96,2.0,Hi everyone! Hope you're having a profitable and problem free week. One of my customers approached me because their suppliers are having a hard time sourcing food grade liquid bulk carriers.\n\nPosted capacity on DAT looks bad. Using the load boards doesnt seem like a reliable way to source trucks. Is there another place I should be looking? What is the best way to identify and engage motor carriers who can move liquid bulk? I currently have shipments going from the midwest to west coast and west coast to Winnipeg.,FreightBrokers,Food Grade Liquid Bulk,2.0,1.00,['Winnipeg']
4,1.633525e+09,0.0,t3_q2juox,1.0,"Just got an email from a carrier, saying, FFE the receiver saying that they cannot receive the load until they get a Electronic Transmission.\n\nAnyone know what they mean by this?",FreightBrokers,Electronic Transmission?,1.0,1.00,[]


In [ ]:
#dropping some duplicates
df = df.drop_duplicates()

In [ ]:
# get sentiment
df['sentiment'] = df['selftext'].apply(get_sentiment)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,gpe,sentiment
0,1.633555e+09,0.0,t3_q2tok0,1.0,"USDOT Number: 3300318, MC-1046672 \n\nThinking of taking my talents elsewhere and lease to them. What they are saying sounds great but you have to hear it from the people who have say to load the truck or not.\n\nSo what’s your point of view?",FreightBrokers,What do you guys think of Cloud Trucks,1.0,1.00,[],NEGATIVE (0.9967)
1,1.633554e+09,0.0,t3_q2teic,1.0,Does anyone use this load board app? I recently starting using it but idk how trustworthy it is,FreightBrokers,Doft?,1.0,1.00,[],POSITIVE (0.6827)
2,1.633546e+09,0.0,t3_q2qvl2,3.0,"I’ve have been applying to various 3PLs in hopes of getting hired as a “carrier sales rep” as most companies call it. I’ve applied to several and none have gotten back to me in almost a week... except for one. I applied to a “certain” company with a... less than stellar reputation around here, and they responded. Now, should they offer me a position I’ll be tempted to take the job and do what I can to learn the business before moving on.\n\nI just wanted to hear you guy’s thoughts on this. Would I be shooting myself in the foot? Should I wait longer to hear back from other companies? Or will everything be fine if I do my best, put in my time, and move on?",FreightBrokers,I guess this might be my only option?,3.0,0.81,[],NEGATIVE (0.9999)
3,1.633529e+09,0.0,t3_q2ky96,2.0,Hi everyone! Hope you're having a profitable and problem free week. One of my customers approached me because their suppliers are having a hard time sourcing food grade liquid bulk carriers.\n\nPosted capacity on DAT looks bad. Using the load boards doesnt seem like a reliable way to source trucks. Is there another place I should be looking? What is the best way to identify and engage motor carriers who can move liquid bulk? I currently have shipments going from the midwest to west coast and west coast to Winnipeg.,FreightBrokers,Food Grade Liquid Bulk,2.0,1.00,['Winnipeg'],NEGATIVE (0.9925)
4,1.633525e+09,0.0,t3_q2juox,1.0,"Just got an email from a carrier, saying, FFE the receiver saying that they cannot receive the load until they get a Electronic Transmission.\n\nAnyone know what they mean by this?",FreightBrokers,Electronic Transmission?,1.0,1.00,[],NEGATIVE (0.9999)


In [ ]:
import ast

df['gpe'] = df['gpe'].apply(lambda x: ast.literal_eval(x))

In [ ]:
# initialize sentiment dictionary
sentiment = {}

# loop through dataframe and extract gpe labels and sentiment scores into sentiment dictionary
for i, row in df.iterrows():
    # extract sentiment direction and score
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # loop through each label in organizations column
    for gpe in row['gpe']:
        # check if org label exists in sentiment dictionary already
        if gpe not in sentiment.keys():
            # if it doesn't, initialize new entry in dictionary
            sentiment[gpe] = {'POSITIVE': [], 'NEGATIVE': []}
        # append positive/negative score to respective dictionary entry
        sentiment[gpe][direction].append(score)

In [ ]:
sentiment['Chicago']


{'NEGATIVE': [0.9996306896209717,
  0.9922383427619934,
  0.9723836779594421,
  0.9948403239250183,
  0.999459445476532,
  0.9430989623069763,
  0.9996306896209717,
  0.9978172779083252],
 'POSITIVE': [0.9919384121894836,
  0.5617364645004272,
  0.7392508387565613,
  0.5617364645004272,
  0.9919384121894836]}

In [ ]:
# initialize sentiment list
# getting average sentiment for GPE
avg_sentiment = []

# loop through each gpe
for gpe in sentiment.keys():
    # get number of positive and negative ratings
    freq = len(sentiment[gpe]['POSITIVE']) + len(sentiment[gpe]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # assign to variable for cleaner code
        score = sentiment[gpe][direction]
        # if there are no entries, set to 0
        if len(score) == 0:
            sentiment[gpe][direction] = 0.0
        else:
            # otherwise calculate total
            sentiment[gpe][direction] = sum(score)
    # now calculate total amount
    total = sentiment[gpe]['POSITIVE'] - sentiment[gpe]['NEGATIVE']
    # and the average score
    avg = total/freq
    # add to sentiment list
    avg_sentiment.append({
        'entity': gpe,
        'positive': sentiment[gpe]['POSITIVE'],
        'negative': sentiment[gpe]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [ ]:
sentiment_df = pd.DataFrame(avg_sentiment)
sentiment_df.head()

,entity,positive,negative,frequency,score
0,Winnipeg,0.000000,0.992479,1,-0.992479
1,Houston,3.729265,7.825486,12,-0.341352
2,Dallas,0.000000,6.573751,7,-0.939107
3,Oregon,1.990452,0.000000,2,0.995226
4,US,8.500751,16.993488,27,-0.314546


In [ ]:
#gpes with counts over 3 are all actual gpes besides "Vlads"
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
1,Houston,3.729265,7.825486,12,-0.341352
2,Dallas,0.000000,6.573751,7,-0.939107
4,US,8.500751,16.993488,27,-0.314546
7,Texas,1.639783,6.943607,9,-0.589314
10,China,0.000000,8.377336,9,-0.930815
11,Florida,0.000000,9.847488,10,-0.984749
16,Montana,0.000000,3.929865,4,-0.982466
20,Chicago,3.846601,7.899099,13,-0.311731
25,California,5.517444,7.788851,14,-0.162243
30,Miami,0.518635,5.990066,7,-0.781633


In [ ]:
sentiment_df.sort_values('score', ascending=False).head(10)


,entity,positive,negative,frequency,score
100,Omaha,3.926332,0.000000,4,0.981583
39,Atlanta,5.669883,2.994209,9,0.297297
40,Vlads,3.726962,2.993497,7,0.104781
45,Memphis,1.785325,1.994852,4,-0.052382
48,Mexico,1.602367,1.978585,4,-0.094055
66,IL,2.982341,3.996678,7,-0.144905
25,California,5.517444,7.788851,14,-0.162243
60,FL,1.979066,2.992966,5,-0.202780
20,Chicago,3.846601,7.899099,13,-0.311731
4,US,8.500751,16.993488,27,-0.314546


In [ ]:
df['gpe_string'] = [','.join(map(str, l)) for l in df['gpe']]


In [ ]:
df_trimmed = df[['selftext', 'gpe_string', 'sentiment']]

In [ ]:
pd.set_option('max_colwidth', 1000)

In [ ]:
atlanta = df_trimmed[df_trimmed['gpe_string'].str.contains("Atlanta")]

In [ ]:
atlanta

,selftext,gpe_string,sentiment
154,"Hi Vlads, Chads, and Freight Dads! I have received an inquiry from a newer brokerage and wanted to know if anyone can provide any possible feedback on them. The company name is Value Logistics Inc (MC 1013935-B) based in Atlanta, GA. I am located in Canada so don't really search up American brokerages and usually work with local brokers. All and any information is appreciated. \n\nTake Care, Love Y'all!","Atlanta,Vlads,Canada",POSITIVE (0.9984)
174,"I quit a large 3PL a few years ago, got burned out by the hours and didn’t really like desk life. I got into bartending/serving at restaurants then got absolutely smoked when Covid hit so I had to get into construction, which I do enjoy but it’s taking a toll on my body.\n\nI’m thinking about getting back into brokering, at a smaller growing company with an office based out of Atlanta GA. I have a strong book of business that I’ve stayed in contact with while I’ve been away, my absolute best carriers and people I’ve actually been good friends with.\n\nI know this market favors the carrier but I have reliable trucks that take good rates. I’ve never worked at an up and coming brokerage so I’d like some advice on what I should expect if I take this job. I’m only used to the big 3PL with 10,000+ loads per day to work with, this company is in the hundreds per day.\n\nI want to make good money and I’m prepared to make less for a while, but I really don’t have any experience with smaller ...",Atlanta,POSITIVE (0.9119)
220,"After reading all the posts about crappy carriers and double brokering I want to ask if I’m the asshole for cancelling a load? I called a broker in Cali and wanted the load but they were unsure if shipper would move my pallet to make it work. I drop my load and the wanted load is gone. No big deal, no rate con signed. Few days later I sign a rate con for Atlanta to PA for $850. Get a better offer for double rate and give broker 4 days notice of cancellation. They leave a bad review on carrier411 about cancelling multiple loads. Not sure what to do.",Atlanta,NEGATIVE (1.0)
274,"Dig deep into the company find its locations and once you find a contact (Jim) \n\n“Hey Jim I have a truck sitting in Atlanta right by your facility today that I need to move, any freight you could use a truck for? \n\nProviding a current solution to a possible problem. Even if they don’t maybe Jim emails you back next time they need a truck and you now have opportunity to get your foot in the door. \n\n#BEcreative",Atlanta,POSITIVE (0.8922)
474,"Vlad here. Through coercion and bribery I was able to get my hands on the TQL beginner's manual.\n\nI want to share some selected chapters:\n\n**Chapter1: People skills**\n\n\-A TQL broker must remain professional no matter what! When a carrier calls and tells you with a hard to understand foreign accent to engage in a reproductive behaviour with you maternal parent - you should explain that you'll pay him a generous $150 TONU for the 8hrs he/she spent at the pick up. There's no way you could've know that the load shipped yesterday! Except maybe calling your customer, but that's discouraged in our company. Charges may apply if you actually do it!\n\n\-When you call a carrier and he/she refuses the load you are pushing - ask them **""WHY!?**"". **POSSIBILITY1:** If the carrier has the misfortune to actually politely proceed &amp; explain - hang up quickly! Call them 5min later and pretend this is the 1st time you call.\n\n**POSSIBILITY2:** If they refuse to explain and hang up on you ...","Atlanta,NE,Omaha,Kansas City,Charleston",POSITIVE (0.9876)
513,"Is anyone hearing anything from carriers in the SE about issues with diesel yet?\n\nI’ve been reading in r/Charlotte and r/Atlanta that regular fuel IS actively being hoarded, so I’m starting to worry it will eventually cause problems for us. \n\nThree of the account managers I schedule for ship in potentially affected areas (SC, NC, GA), and I keep telling them not to acc